In [1]:
!pip install transformers
!pip install requests


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

model_name = "savasy/bert-base-turkish-sentiment-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ["Negatif", "Nötr", "Pozitif"]

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
    pred = torch.argmax(probs).item()
    return labels[pred], probs[0].tolist()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [3]:
import requests

TMDB_API_KEY = "6899ac0ff0fb9dbbf05efbec9c8187b2"
TMDB_API_URL = "https://api.themoviedb.org/3"

# Ruh haline göre film türü eşleştirmesi
emotion_to_genre = {
    "Pozitif": 35,     # Komedi
    "Negatif": 18,     # Dram
    "Nötr": 10749      # Romantik
}

def get_movies_by_genre(genre_id):
    url = f"{TMDB_API_URL}/discover/movie"
    params = {
        "api_key": TMDB_API_KEY,
        "with_genres": genre_id,
        "sort_by": "popularity.desc",
        "language": "tr-TR"
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data["results"][:5]  # En popüler 5 film

def film_öner(emotion):
    genre_id = emotion_to_genre[emotion]
    movies = get_movies_by_genre(genre_id)
    print(f"🎬 {emotion} ruh hali için film önerileri:\n")
    for movie in movies:
        print(f"- {movie['title']} ({movie['release_date'][:4]})\n  {movie['overview']}\n")


In [6]:
user_input = input("📥 Bugün nasıl hissediyorsun? ")

duygu, olasılıklar = analyze_sentiment(user_input)
print(f"\n🔍 Tespit edilen duygu: {duygu}")
print(f"Olasılıklar: {dict(zip(labels, [round(p, 3) for p in olasılıklar]))}")

film_öner(duygu)


📥 Bugün nasıl hissediyorsun? okula gittim çok kötü bi gündü berbat

🔍 Tespit edilen duygu: Negatif
Olasılıklar: {'Negatif': 0.999, 'Nötr': 0.001}
🎬 Negatif ruh hali için film önerileri:

- Brave Citizen (2023)
  Kahraman So Si Min’in hikayesi, liseli sporcu olarak üst düzey hapkido becerileriyle ülkede tanındığı dönemi anlatıyor. Yedi yıl sonra, yarı zamanlı bir öğretmen olarak hayatına devam ederken, So Si Min birden bire kötü şöhreti olan öğrenci Han Soo Kang’ın hedefi haline gelir.

- Lost Bullet 3 (2025)
  Üçlemenin bu turbo hızlı finalinde araba dâhisi Lino, Areski'den ve hayatlarını mahveden yozlaşmış kumandandan intikamını almak için geri dönüyor.

- Conjuring the Cult (2024)
  

- Karate Kid: Efsane Dövüşçüler (2025)
  Pekin'den New York'a taşınan genç kung fu yeteneği Li Fong'un hikâyesi anlatılıyor. Ailevi bir trajedinin ardından yeni hayatına uyum sağlamaya çalışan Li, bir arkadaşına yardım etmek için bir karate turnuvasına katılmaya karar verir. Ancak, kung fu bilgisi yeter